# This notebook provides a guideline how to create your own bidder class and to check it on our datasets.

In [ ]:
import sys
sys.path.append("../")

In [ ]:
import optuna
import pandas as pd
import numpy as np
from typing import Dict

import matplotlib.pyplot as plt

from simulator.simulation.modules import Campaign, History
from simulator.simulation.simulate import simulate_campaign
from simulator.simulation.utils_visualization import data_prep_vis, plot_history_article
from simulator.model.bidder import _Bidder
from simulator.validation.check_results import autobidder_check

### Create bidder with a simple bidding strategy

In [ ]:
class AllInBidder(_Bidder):
    """
    A baseline bidder with a simple bidding strategy.

    This class serves as an example of a working bidder and can be used
    as a template for more sophisticated bidding strategies.

    Attributes:
        Any custom attributes can be added here.
    """

    def __init__(self, params: dict = None):
        super().__init__()

    def place_bid(self, bidding_input_params: Dict[str, any], history: History) -> float:
        """
        Calculate and return a bid based on the campaign's balance.

        This method demonstrates the use of input parameters to determine the bid.

        Args:
            bidding_input_params: A dictionary containing bidding input parameters.
            history: Auction history for the specific campaign (unused in this implementation).

        Returns:
            float: The calculated bid amount.
        """
        
        bid = bidding_input_params["balance"]
        return bid

### Let's check our risky bidder on one campaign's data

In [ ]:
auction_mode = 'FPA' # you may also choose the 'VCG' mode
bidder_class = AllInBidder

# data paths
if auction_mode == 'FPA':
    campaigns_path = '../data/campaign_example_fpa.csv'
    stats_path = '../data/stats_example_fpa.csv'
elif auction_mode == 'VCG':
    campaigns_path = ''
    stats_path = ''

In [ ]:
campaign_df = pd.read_csv(campaigns_path)
stats_df = pd.read_csv(stats_path)

In [ ]:
CAMPAIGN_IDX = 5

In [ ]:
campaign_data = campaign_df.iloc[CAMPAIGN_IDX]
campaign_example = Campaign(
    item_id=campaign_data['item_id'],
    campaign_id=int(campaign_data['campaign_id']),
    loc_id=int(campaign_data["loc_id"]),
    region_id=int(campaign_data["region_id"]),
    logical_category=campaign_data["logical_category"],
    microcat_ext=int(campaign_data["microcat_ext"]),
    campaign_start=int(campaign_data["campaign_start"]),
    campaign_end=int(campaign_data["campaign_end"]),
    initial_balance=campaign_data['auction_budget'],
    balance=campaign_data['auction_budget'],
    curr_time=int(campaign_data["campaign_start"]),
    prev_time=int(campaign_data["campaign_start"]),
    prev_balance=campaign_data['auction_budget'],
)
campaign_data['auction_budget'], (campaign_data['campaign_end'] - campaign_data['campaign_start']) // 3600

In [ ]:
simulation_history = simulate_campaign(
    campaign=campaign_example,
    bidder=bidder_class(),
    stats_file=stats_df,
    auction_mode=auction_mode
)

# Visualization of one campaign's simulation history

In [ ]:
df = simulation_history.to_data_frame()
vis_data = data_prep_vis(df)
df.head(2)

In [ ]:
fig = plot_history_article(vis_data)

# Metrics check

In [ ]:
res = autobidder_check(
        bidder=AllInBidder,
        params={
            "input_campaigns": campaigns_path,
            "input_stats": stats_path,
        },
        auction_mode=auction_mode
    )

print(f"MCR: {res['score'][0]}, RMSE_T: {res['score'][1]}")

As you can see, this bidder's performance leaves room for improvement. Good luck with your own bidders, and may your bids be ever in your favor!